# Notebook for making Ungulate Threat to GDEs

Name:        Threats to GDEs due to ungulates

Purpose:     Determine whether GDEs are at risk of damage from ungulates (depends on GDE type and ungulate potential presence)
              
Refer to spreadsheet for notes on affected systems:  D:\GDE_Threats\OtherData\Grazing\grazing_impacts_by_GDE.xlsx

GDE methods report has descriptions of systems for each layer: https://www.conservationgateway.org/ConservationByGeography/NorthAmerica/UnitedStates/nevada/water/Documents/NV_iGDE_MethodsReport.pdf

Author:      sarah.byer


## Set up environments in ArcGIS Pro

In [1]:
# Import ArcGIS modules and check out spatial analyst extension
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.sa import *
arcpy.CheckOutExtension("spatial")

'CheckedOut'

In [2]:
# Path to temporary geodatabase
#path =  r"folder_path\gdbname.gdb"
path =  r"K:\GIS3\Projects\GDE\Maps\GDE_Threats\GDE_Threats.gdb"

# Environment settings
env.workspace = path
env.overwriteOutput = True
env.outputCoordinateSystem = arcpy.SpatialReference(26911) # Spatial reference NAD 1983 UTM Zone 11N. The code is '26911'

In [3]:
# Custom function that converts tables to pandas dataframe

def table_to_data_frame(in_table, input_fields=None, where_clause=None):
    """Function will convert an arcgis table into a pandas dataframe with an object ID index, and the selected
    input fields using an arcpy.da.SearchCursor."""
    OIDFieldName = arcpy.Describe(in_table).OIDFieldName
    if input_fields:
        final_fields = [OIDFieldName] + input_fields
    else:
        final_fields = [field.name for field in arcpy.ListFields(in_table)]
    data = [row for row in arcpy.da.SearchCursor(in_table, final_fields, where_clause=where_clause)]
    fc_dataframe = pd.DataFrame(data, columns=final_fields)
    fc_dataframe = fc_dataframe.set_index(OIDFieldName, drop=True)
    return fc_dataframe

### Read in ungulate data

BLM National Grazing Allotments: https://gbp-blm-egis.hub.arcgis.com/datasets/BLM-EGIS::blm-natl-grazing-allotment-polygons/explore

BLM National Wild Horse and Burro: https://gbp-blm-egis.hub.arcgis.com/datasets/BLM-EGIS::blm-nv-wild-horse-and-burro-herd-area-polygons/about 

NDOW Occupied Elk Distribution: https://gis-ndow.opendata.arcgis.com/datasets/NDOW::ndow-big-game-distributions/about?layer=2

In [4]:
# Copy features to geodatabase

#graze = arcpy.CopyFeatures_management(r'path_to_folder\BLM_National_Grazing_Allotments\gra.gdb\gra_allot_poly', 'graze') # Cattle/sheep grazing allotments
#hma = arcpy.CopyFeatures_management(r'path_to_folder\BLM_National_Wild_Horse_and_Burro\blm_natl_whb_geocortex.gdb\whb_hma_pop_poly', 'hma') # Horse/burro
#elk = arcpy.CopyFeatures_management(r'path_to_folder\Occupied_Elk_Distribution.shp', 'elk') # Elk habitat

graze = arcpy.CopyFeatures_management(r'K:\GIS3\Data_Library\National\Management\Grazing\BLM_National_Grazing_Allotments\gra.gdb\gra_allot_poly', 'graze')
hma = arcpy.CopyFeatures_management(r'K:\GIS3\Data_Library\National\Management\Horse_Burro\blm_natl_whb_geocortex.gdb\whb_hma_pop_poly', 'hma')
elk = arcpy.CopyFeatures_management(r'K:\GIS3\Data_Library\NV\Biota\Occupied_Elk_Distribution.shp', 'elk')


KeyboardInterrupt: 

### Read in GDE data

Only allow ungulates to affect phreatophyte communities, wetlands, and springs.

In [ ]:
# Load features from iGDE database
#phr = r'path_to_folder\NV_iGDE_MMDDYY.gdb\Phreatophytes' # NOTE - This feature should be "exploded" first (multipart to singlepart)
#wet = r'path_to_folder\NV_iGDE_MMDDYY.gdb\Wetlands'
#springs = r'path_to_folder\NV_iGDE_MMDDYY.gdb\Springs'
#rs = r'path_to_folder\NV_iGDE_MMDDYY.gdb\Rivers_Streams'
#lp = r'path_to_folder\NV_iGDE_MMDDYY.gdb\Lakes_Playas'

phr = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess.gdb\Phreatophyte_Explode'
wet = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess.gdb\Wetlands'
springs = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess.gdb\Springs'
lp = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess.gdb\Lakes_Playas'
rs = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess.gdb\Rivers_Streams'

In [6]:
# Hydro areas and hexagons for summarizing
# Only copy once, then read in

#ha = arcpy.CopyFeatures_management(r'path_to_folder\NV_HydrographicAreas.shp', 'ha_ungulates')
#hexagons = arcpy.CopyFeatures_management(r'path_to_folder\NV_Hexagons.shp', 'hex_ungulates')

ha = arcpy.CopyFeatures_management(r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_Story_041522.gdb\NV_HydrographicAreas', 'ha_ungulates')
hexagons = arcpy.CopyFeatures_management(r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_Story_041522.gdb\NV_Hexagons', 'hex_ungulates')

ha = 'ha_ungulates'
hexagons = 'hex_ungulates'

In [ ]:
print([f.name for f in arcpy.ListFields(ha)])

OSError: "ha_ungulates" does not exist

## Rules for groups of/individual GDE types

Only specific ungulates count as a threat to specific GDE types

### Phreatophytes - Aspen types

Ungulate threat applies to Aspen Woodland and Aspen Mixed-conifer. These are affected by:
- Cows = graze allot - cows
- Sheep = graze allot - sheep
- Elk = elk distribution

In [ ]:
arcpy.AddField_management(phr, 'GRAZE', 'LONG')
arcpy.AddField_management(phr, 'HORSE_BURRO', 'LONG')
arcpy.AddField_management(phr, 'ELK', 'LONG')
arcpy.CalculateField_management(phr, 'GRAZE', 0, 'PYTHON3')
arcpy.CalculateField_management(phr, 'HORSE_BURRO', 0, 'PYTHON3')
arcpy.CalculateField_management(phr, 'ELK', 0, 'PYTHON3')

In [ ]:
# Impacts from sheep and cow grazing
where = "PHR_TYPE LIKE '%Aspen%'"
aspen = arcpy.SelectLayerByAttribute_management(phr, 'NEW_SELECTION', where) # Select aspen
print(int(arcpy.GetCount_management(aspen)[0]))
aspen_select = arcpy.SelectLayerByLocation_management(aspen, 'INTERSECT', graze, '', 'SUBSET_SELECTION') # sub-select grazing
print(int(arcpy.GetCount_management(aspen_select)[0]))
arcpy.CalculateField_management(aspen_select, 'GRAZE', 1, 'PYTHON3')

In [ ]:
# Elk distribution
aspen = arcpy.SelectLayerByAttribute_management(phr, 'NEW_SELECTION', where) # Select aspen
aspen_select = arcpy.SelectLayerByLocation_management(aspen, 'INTERSECT', elk, '', 'NEW_SELECTION') # sub-select elk
print(int(arcpy.GetCount_management(aspen_select)[0]))
arcpy.CalculateField_management(aspen_select, 'ELK', 1, 'PYTHON3')

In [ ]:
# Sum up ungulate impacts
arcpy.AddField_management(phr, 'UNGULATES', 'LONG')
arcpy.CalculateField_management(phr, 'UNGULATES', '!GRAZE! + !HORSE_BURRO! + !ELK!', 'PYTHON3')

### Phreatophytes - Mesquite

Mesquite affected by:
- Horse/burros
- Cows - in the Mojave if grazing permits are not retired

At the time of analysis, the following allotments were active:
- MORMON PEAK NV01044
- ROX-TULE NV01063
- GARDEN SPRING NV01065
- GOURD SPRING NV01071
- SNOW SPRINGS NV01074
- SUMMIT SPRING NV01077
- WHITE ROCK NV01078
- PAHRANAGAT WEST NV01081
- DELAMAR NV01083
- BREEDLOVE NV11010
- LOWER LAKE WEST NV11013
- GRAPEVINE NV11032
- HENRIE COMPLEX NV11034
- BALD MTN NV21003
- LIME MOUNTAIN NV21005
- LOWER LAKE EAST NV21022
- Hidden Valley NV15412
- CRESCENT (N-4) NV00128
- RAZORBACK NV00093
- MACGRUDER MTN NV00099

In [ ]:
# Horse/burro distribution
where = "PHR_TYPE LIKE '%Mesquite%'"
mesquite = arcpy.SelectLayerByAttribute_management(phr, 'NEW_SELECTION', where) # Select mesquite
int(arcpy.GetCount_management(mesquite)[0]) 
mesquite_select = arcpy.SelectLayerByLocation_management(mesquite, 'INTERSECT', hma, '', 'SUBSET_SELECTION')
int(arcpy.GetCount_management(mesquite_select)[0]) 
arcpy.CalculateField_management(mesquite_select, 'HORSE_BURRO', 1, 'PYTHON3')

In [ ]:
# Cow grazing in the Mojave - need to subset grazing allotment data
# Subset non-retired grazing permits (mesquite only in the mojave anyway)
# Check authorization use by allotment forms for any active allotment that overlaps with mapped mesquite
mesquite = arcpy.SelectLayerByAttribute_management(phr, 'NEW_SELECTION', where) # Select mesquite
graze_select = arcpy.SelectLayerByLocation_management(graze, 'INTERSECT', mesquite, '', 'NEW_SELECTION')
int(arcpy.GetCount_management(graze_select)[0]) 
allots = arcpy.CopyFeatures_management(graze_select, 'mesquite_graze_allots')
allots = 'mesquite_graze_allots'
[f.name for f in arcpy.ListFields(allots)]

active = ['NV01044', 'NV01063', 'NV01065', 'NV01071', 'NV01074', 'NV01077',
'NV01078', 'NV01081', 'NV01083', 'NV11010', 'NV11013', 'NV11032',
'NV11034', 'NV21003', 'NV21005', 'NV21022', 'NV15412', 'NV00128',
'NV00093', 'NV00099']

with arcpy.da.UpdateCursor(allots, ['ST_ALLOT']) as cursor:
    for row in cursor:
        if row[0] in active:
            print(row[0])
        else:
            cursor.deleteRow()
del cursor

where = "PHR_TYPE LIKE '%Mesquite%'"
mesquite = arcpy.SelectLayerByAttribute_management(phr, 'NEW_SELECTION', where) # Select mesquite
int(arcpy.GetCount_management(mesquite)[0]) 
mesquite_select = arcpy.SelectLayerByLocation_management(mesquite, 'INTERSECT', allots, '', 'SUBSET_SELECTION')
int(arcpy.GetCount_management(mesquite_select)[0]) 
arcpy.CalculateField_management(mesquite_select, 'GRAZE', 1, 'PYTHON3')



In [ ]:
# Sum up ungulate impacts in PHREATOPHYTES
arcpy.CalculateField_management(phr, 'UNGULATES', '!GRAZE! + !HORSE_BURRO! + !ELK!', 'PYTHON3')

### Wetlands

Wet meadows and montane riparian == meadow; palustrine forests == pf. These are all affected by:
- Horse/burros
- Cows/sheep

Currently we have ovevrlap between meadow and pf features. Need to refine meadow selection - can't include 'palustrine forest'

In [ ]:
arcpy.AddField_management(wet, 'GRAZE', 'LONG')
arcpy.AddField_management(wet, 'HORSE_BURRO', 'LONG')
arcpy.AddField_management(wet, 'ELK', 'LONG')
arcpy.CalculateField_management(wet, 'GRAZE', 0, 'PYTHON3')
arcpy.CalculateField_management(wet, 'HORSE_BURRO', 0, 'PYTHON3')
arcpy.CalculateField_management(wet, 'ELK', 0, 'PYTHON3')
print([f.name for f in arcpy.ListFields(wet)])

In [ ]:
# Palustrine - Aquatic, Emergent, Meadow
# Affected by cow, sheep, and horse/burrow
# Impacts from sheep and cow grazing
where = "WET_TYPE = 'Palustrine' AND WET_SUBTYPE IN ('aquatic', 'emergent', 'meadow')"
aqua = arcpy.SelectLayerByAttribute_management(wet, 'NEW_SELECTION', where) # Select aquatic, emergent, meadow
int(arcpy.GetCount_management(aqua)[0]) 
aqua_select = arcpy.SelectLayerByLocation_management(aqua, 'INTERSECT', graze, '', 'SUBSET_SELECTION') # sub-select grazing
int(arcpy.GetCount_management(aqua_select)[0]) 
arcpy.CalculateField_management(aqua_select, 'GRAZE', 1, 'PYTHON3')

# Impacts from horse/burrow
aqua = arcpy.SelectLayerByAttribute_management(wet, 'NEW_SELECTION', where) # Select aquatic, emergent, meadow
int(arcpy.GetCount_management(aqua)[0]) 
aqua_select = arcpy.SelectLayerByLocation_management(aqua, 'INTERSECT', hma, '', 'SUBSET_SELECTION') # sub-select HMAs
int(arcpy.GetCount_management(aqua_select)[0]) 
arcpy.CalculateField_management(aqua_select, 'HORSE_BURRO', 1, 'PYTHON3')

In [ ]:
# Riparian - montane
# Cows and sheep
where = "WET_TYPE = 'Riparian' AND WET_SUBTYPE = 'montane'"
aqua = arcpy.SelectLayerByAttribute_management(wet, 'NEW_SELECTION', where) # Select montane riparian
int(arcpy.GetCount_management(aqua)[0]) 
aqua_select = arcpy.SelectLayerByLocation_management(aqua, 'INTERSECT', graze, '', 'SUBSET_SELECTION') # sub-select grazing
int(arcpy.GetCount_management(aqua_select)[0]) 
arcpy.CalculateField_management(aqua_select, 'GRAZE', 1, 'PYTHON3')

# Horse and burro
aqua = arcpy.SelectLayerByAttribute_management(wet, 'NEW_SELECTION', where) # Select montane riparian
aqua_select = arcpy.SelectLayerByLocation_management(aqua, 'INTERSECT', hma, '', 'SUBSET_SELECTION') # sub-select grazing
int(arcpy.GetCount_management(aqua_select)[0]) 
arcpy.CalculateField_management(aqua_select, 'HORSE_BURRO', 1, 'PYTHON3')

In [ ]:
# Sum up ungulate impacts in WETLANDS
arcpy.AddField_management(wet, 'UNGULATES', 'LONG')
arcpy.CalculateField_management(wet, 'UNGULATES', '!GRAZE! + !HORSE_BURRO! + !ELK!', 'PYTHON3')

### Springs

Affected by:
- Horse/burros
- Cows/sheep

Not considering whether springs are fenced - data not available on statewide scales

In [ ]:
arcpy.AddField_management(springs, 'GRAZE', 'LONG')
arcpy.AddField_management(springs, 'HORSE_BURRO', 'LONG')
arcpy.AddField_management(springs, 'ELK', 'LONG')
arcpy.CalculateField_management(springs, 'GRAZE', 0, 'PYTHON3')
arcpy.CalculateField_management(springs, 'HORSE_BURRO', 0, 'PYTHON3')
arcpy.CalculateField_management(springs, 'ELK', 0, 'PYTHON3')
print([f.name for f in arcpy.ListFields(springs)])

In [ ]:
# Attribute springs affected by horses/burrows and/or sheep/cows
# Sheep/cows
spr_select = arcpy.SelectLayerByLocation_management(springs, 'INTERSECT', graze, '', 'NEW_SELECTION') # sub-select grazing
int(arcpy.GetCount_management(spr_select)[0]) 
arcpy.CalculateField_management(spr_select, 'GRAZE', 1, 'PYTHON3')

# Horses/burros
spr_select = arcpy.SelectLayerByLocation_management(springs, 'INTERSECT', hma, '', 'NEW_SELECTION') # sub-select HMAs
int(arcpy.GetCount_management(spr_select)[0]) 
arcpy.CalculateField_management(spr_select, 'HORSE_BURRO', 1, 'PYTHON3')

In [ ]:
# Sum up ungulate impacts in SPRINGS
arcpy.AddField_management(springs, 'UNGULATES', 'LONG')
arcpy.CalculateField_management(springs, 'UNGULATES', '!GRAZE! + !HORSE_BURRO! + !ELK!', 'PYTHON3')

### Rivers & Streams

Affected by:
- Horse/burros
- Cows/sheep

In [ ]:
arcpy.AddField_management(rs, 'GRAZE', 'LONG')
arcpy.AddField_management(rs, 'HORSE_BURRO', 'LONG')
arcpy.AddField_management(rs, 'ELK', 'LONG')
arcpy.CalculateField_management(rs, 'GRAZE', 0, 'PYTHON3')
arcpy.CalculateField_management(rs, 'HORSE_BURRO', 0, 'PYTHON3')
arcpy.CalculateField_management(rs, 'ELK', 0, 'PYTHON3')
[f.name for f in arcpy.ListFields(rs)]

In [ ]:
# Attribute rivers/streams affected by horses/burrows and/or sheep/cows
# Sheep/cows
rs_select = arcpy.SelectLayerByLocation_management(rs, 'INTERSECT', graze, '', 'NEW_SELECTION') # sub-select grazing
int(arcpy.GetCount_management(rs_select)[0]) 
arcpy.CalculateField_management(rs_select, 'GRAZE', 1, 'PYTHON3')

# Horses/burros
rs_select = arcpy.SelectLayerByLocation_management(rs, 'INTERSECT', hma, '', 'NEW_SELECTION') # sub-select HMAs
int(arcpy.GetCount_management(rs_select)[0]) 
arcpy.CalculateField_management(rs_select, 'HORSE_BURRO', 1, 'PYTHON3')

In [ ]:
# Sum up ungulate impacts in SPRINGS
arcpy.AddField_management(rs, 'UNGULATES', 'LONG')
arcpy.CalculateField_management(rs, 'UNGULATES', '!GRAZE! + !HORSE_BURRO! + !ELK!', 'PYTHON3')

### Lakes & Playas

Not affected by ungulates

In [ ]:
arcpy.AddField_management(lp, 'UNGULATES', 'LONG')
arcpy.CalculateField_management(lp, 'UNGULATES', 0, 'PYTHON3')

## Normalize Ungulate Scores

Ungulate score is a STRESSOR only - it's a current impact.

Divide UNGULATES value by 2 - the maximum score of ungulate impacts for any feature.


In [ ]:
# GDE layers
#phr = r'path_to_folder\NV_iGDE_MMDDYY.gdb\Phreatophytes'
#wet = r'path_to_folder\NV_iGDE_MMDDYY.gdb\Wetlands'
#springs = r'path_to_folder\NV_iGDE_MMDDYY.gdb\Springs'
#rs = r'path_to_folder\NV_iGDE_MMDDYY.gdb\Rivers_Streams'
#lp = r'path_to_folder\NV_iGDE_MMDDYY.gdb\Lakes_Playas'

phr = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess.gdb\Phreatophyte_Explode'
wet = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess.gdb\Wetlands'
springs = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess.gdb\Springs'
lp = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess.gdb\Lakes_Playas'
rs = r'K:\GIS3\Projects\GDE\Geospatial\NV_iGDE_assess.gdb\Rivers_Streams'

arcpy.AddField_management(springs, 'UnguStr', 'DOUBLE')
arcpy.CalculateField_management(springs, 'UnguStr', '!UNGULATES!/2', 'PYTHON3')

arcpy.AddField_management(wet, 'UnguStr', 'DOUBLE')
arcpy.CalculateField_management(wet, 'UnguStr', '!UNGULATES!/2', 'PYTHON3')

arcpy.AddField_management(phr, 'UnguStr', 'DOUBLE')
arcpy.CalculateField_management(phr, 'UnguStr', '!UNGULATES!/2', 'PYTHON3')

arcpy.AddField_management(rs, 'UnguStr', 'DOUBLE')
arcpy.CalculateField_management(rs, 'UnguStr', '!UNGULATES!/2', 'PYTHON3')

arcpy.AddField_management(lp, 'UnguStr', 'DOUBLE')
arcpy.CalculateField_management(lp, 'UnguStr', '!UNGULATES!/2', 'PYTHON3')